## Movie Reviews Sentiment Analysis 

This notebook is a step by step walk-through on how to train a simple naive bayes classfier to recognize wether it the film has received a **positive** or **negative** sentiment

Before we start, below is a simple naive bayes equation we will mainly be using for calculating the probabilities.
1. Prior: number of files in given class, i.e. if 2 out of 5 reviews are positive, 0.4 will be its prior prob.
2. Likelihood or P(feature|class): give num of features, what's the likelihood that its a good film ((i.e. good, amazing, awesome)|positive)
3. Evidence: number of data points (here namely reviews) we have. 

Note that in this exercise about computing the denominator for the naive Bayes classifier, we can ignore the denominator since we're comparing P(positive|reviews) and P(negative|reviews) and so can cancel out their denominators to simplify our work.

I have also attached a picture of the pseudocode demonstration from Juravsky& Martin's SLP textbook.

![png](./nb-pseudocode.png)

## Building and Storing Feature Vectors

Create parameters to store the **features** into an appropriate data structure of your choice. 

Here `numpy` is used to create matrices for creating **feature vectors**
In the past, I have primarily used `dictionaries` for storing data. Alternatively, `numpy` supports various magic operations on the data structure and is very powerful. Therefore, here `numpy` is used.

Please click for more information about how to use [numpy](https://cs231n.github.io/python-numpy-tutorial/).



In [18]:
import os
import numpy as np
from collections import Counter, defaultdict
import nltk

# nltk.download()
# nltk.download('punkt')

class NaiveBayes():

    def __init__(self):
        # be sure to use the right class_dict for each data set
        self.class_dict = {0: 'pos', 1: 'neg'}
        self.feature_dict = defaultdict(Counter)
        self.prior = np.zeros(2)
        self.likelihood = None

Before we can do anything, we first have to load data into our workspace.
The following code scrapes all the roots, dirs, and directories for the files we need and does some basic
text tokenization and counting.

We have two classes `doc_class_1/2` preprocessing or `dict_class_1/2` postprocessing. 

In [19]:
    def preprocessing(self,train_set):
        N_doc = 0 #number of documents
        N_class = np.zeros(2) #num of docs in given class
        big_doc = defaultdict(Counter)
        
        for root, dirs, files in os.walk(train_set):
            for name in files:
                with open(os.path.join(root, name)) as f:
                    tokens = nltk.word_tokenize(f.read())
                    if root == r"Users/shiyishen/Downloads/movie_reviews⁩/train/action":
                        N_class[0] += 1     # 0 represents action class
                        self.feature_dict[0].update(Counter(tokens))
                    else:
                        N_class[1] += 1     # 1 represents comedy class 
                        self.feature_dict[1].update(Counter(tokens))
                    N_doc += 1
                    doc_all.extend(tokens)
         
        
        for i in range(2):
            self.prior[i] = np.log()
        
       


## Training Our NB Classifier 
Remember we have defined three parameters `dict_action`, `dict_comedy`, and `doc_all` in the previous cell. As we have iterated through the data folder and loaded in their corresponding text, what's left is count the number of words that each category and the overall data contain. We'll use NLTK's `.FreqDist` to directly compute their **frequency distribution**.

Now we can then start to create our **feature vectors**. We do it first by creating an array to store all of our features, which are unique words in our training files. Or you could do your own feature selection. 


Now we are done with data preprocessing. Let's head into training our model.

If you still remember the equation. To calculate the NB distribution of a given class, we need its **prior probability** and the **likelikhood** that each feature appears in the class. 
Here we use log space to smoothe our calculation, as we might encounter some significantly small number.

We'll also be using **Laplace's smoothing** technique also called **add-one smoothing**, in which each word appears one extra time. This is to smooth out words with zero probability that might mess up with our likelihood calculation. 

MLE estimate: 
$$ P_{MLE}(w_{i} | w_{i-1}) = \frac{c(w_{i-1},w_{i})}{c(w_{i-1})}$$

Add-1(Laplace) estimate:
$$ P_{Add-1}(w_{i} | w_{i-1}) = \frac{c(w_{i-1},w_{i}) + 1}{c(w_{i-1}) + V}$$

After finishing calculating prior, we can go on calculate the likelihood for all the features. 

In [23]:
    def train(self, train_set):
        for i in range(len(self.class_dict)):
                count_wc = np.zeros(len(self.feature_dict))
                sum_count = len(self.feature_dict)   
                if i == 0: 
                    for j in range(len(self.feature_dict)):
                        if self.feature_dict[j] in dict_class_1.keys():
                            count_wc[j] = dict_class_1[self.feature_dict[j]]
                            sum_count += count_wc[j]
                        else:
                            count_wc[j] = 0
                    for j in range(len(self.feature_dict)):
                        self.likelihood[0][j] = (count_wc[j] + 1) / sum_count
                if i == 1:  
                    for j in range(len(self.feature_dict)):

                        if self.feature_dict[j] in dict_class_2.keys():
                            count_wc[j] = dict_class_2[self.feature_dict[j]]
                            sum_count += count_wc[j]
                        else:
                            count_wc[j] = 0
                    for j in range(len(self.feature_dict)):
                        self.likelihood[1][j] = (count_wc[j] + 1) / sum_count

            self.loglikelihood= np.log(self.likelihood)
            return self.loglikelihood

## Predict and Classify
It's time to predict and classify development set with our newly trained model.

One useful method in numpy is the `.dot` method, which gives the product of **a** and **b**.  

In such case, the higher the likelihood the more confident that it's the given class. 

In [25]:
def test(self, dev_set):
    results = defaultdict(dict)
    for root, dirs, files in os.walk(dev_set):
        for name in files:
            if name!='.DS_Store':
                feature_vector = np.zeros(len(self.feature_dict))
                results[name] = []
                with open(os.path.join(root, name)) as f:
                    text = nltk.word_tokenize(f.read())
                    if root == '/Users/shiyishen/Downloads/movie_reviews/dev/pos':
                        results[name].append('pos')
                    else:
                        results[name].append('neg')

                    dict_text = nltk.FreqDist(text)
                    for i in range(len(self.feature_dict)):
                        if [i] in text:
                            feature_vector[i] = dict_text[self.feature_dict[i]]
                        else:
                            pass

                feature_vector.transpose()
                compare = np.dot(self.likelihood, feature_vector)
                compare = compare + self.logprior
                if compare[0] > compare[1]:
                    results[name].append('pos')
                elif compare[0] < compare[1]:
                    results[name].append('neg')
                else:
                    pass

## Feature Selection 

Congradulations, if you have reached here! Now, you have successfully created your very own Multinomial Naive Bayes Classifier! To top it off with something more interesting, we can virtually do anything for feature selection.

A small tip: 